In [1]:
import os
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2

In [2]:
sr_vis = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0.0, 'max': 0.3,} # for LANDSAT8 surface reflectance RGB

In [3]:
def scaled(image):  # for LANDSAT8 surfance reflectance
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

def select_bands(image): # select bands for training
    opticalBands = image.select('SR_B.')
    thermalRadianceBand = image.select('ST_TRAD')
    return image.addBands(opticalBands, None, True).addBands(thermalRadianceBand, None, True)

List of bands for training: ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_TRAD']

# Defining Boundary for Houston

Format for coordinates:</br>
[latitude, longitude] or [w, h] </br>
in this order: </br>
B C </br>
A D </br>

In [ ]:
BOUNDING_BOX = [[[-95.690165, 29.5937], [-95.690165, 30.266005], [-94.900379, 30.266005], [-94.900379, 29.5937]]]

Map = geemap.Map(center=(29.7662, -95.2410), zoom=9)

sample_tile_scaled = scaled(ee.Image("LANDSAT/LC08/C02/T1_L2/LC08_025039_20210807"))
sample_tile = ee.Image("LANDSAT/LC08/C02/T1_L2/LC08_025039_20210807").select(['SR_B.', 'ST_TRAD'])
polygon = ee.Geometry.Polygon(BOUNDING_BOX)
houston_bounds = ee.Geometry(polygon, None, False)
Map.addLayer(sample_tile_scaled, sr_vis, 'Sample Tile')
Map.addLayer(houston_bounds, {'color': 'FF0000'}, 'planar polygon');
Map

In [ ]:
ee.Initialize()

In [97]:
CROP_WIDTH = .0775  # longitude
CROP_HEIGHT = .0675  # latitude
def random_crop(boundary, idx=0):  # assuming crop size of (256, 256)
    boundary = np.array(boundary)[0]
    # print(boundary)
    w_min, w_max = boundary[:, 0].min(), boundary[:, 0].max()
    h_min, h_max = boundary[:, 1].min(), boundary[:, 1].max()
    
    # draw random point as upper_left corner
    w = np.random.uniform(w_min, w_max - CROP_WIDTH)
    h = np.random.uniform(h_min, h_max - CROP_HEIGHT)
    sample_coord = [[[w, h], [w, h+CROP_HEIGHT], [w+CROP_WIDTH, h+CROP_HEIGHT], [w+CROP_WIDTH, h]]]
    sample_polygon = ee.Geometry.Polygon(sample_coord)
    sample_region = ee.Geometry(sample_polygon, None, False)
    Map.addLayer(sample_region, {'color': 'blue'}, 'sample_{}'.format(idx));
    return sample_region
    
def get_numpy(sample_tile, sample_region):
    image_sample_np = geemap.ee_to_numpy(sample_tile, region=sample_region, default_value=0).astype('float64')
    print("before resize", image_sample_np.shape)
    image_sample_np = cv2.resize(image_sample_np, (256, 256), interpolation=cv2.INTER_LINEAR)
    print("after cv2.INTER_LIENEAR", image_sample_np.shape)
    return image_sample_np
    
# sample_region = random_crop(BOUNDING_BOX)
# sample = get_numpy(sample_tile, sample_region)

In [98]:
for i in tqdm(range(10)):
    sample_region = random_crop(boundary=BOUNDING_BOX, idx=i)
    sample = get_numpy(sample_tile, sample_region)

 10%|█         | 1/10 [00:03<00:31,  3.52s/it]

before resize (256, 255, 8)
after cv2.INTER_LIENEAR (256, 256, 8)


 20%|██        | 2/10 [00:06<00:27,  3.38s/it]

before resize (255, 254, 8)
after cv2.INTER_LIENEAR (256, 256, 8)


 30%|███       | 3/10 [00:10<00:23,  3.32s/it]

...

before resize (256, 255, 8)
after cv2.INTER_LIENEAR (256, 256, 8)


 90%|█████████ | 9/10 [00:30<00:03,  3.34s/it]

before resize (255, 255, 8)
after cv2.INTER_LIENEAR (256, 256, 8)


100%|██████████| 10/10 [00:33<00:00,  3.32s/it]

before resize (254, 254, 8)
after cv2.INTER_LIENEAR (256, 256, 8)


In [138]:
print(sample_tile.bandNames().getInfo())

['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_TRAD']
